In [1]:
import os
import pandas as pd
import numpy as np

import re
import csv
from tqdm.auto import tqdm
import multiprocessing
from functools import partial

import requests
from bs4 import BeautifulSoup

C:\Users\WESLEYQUEST\anaconda3\envs\BigDataUtilization2023\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
def clawringContentImage(href):
    response = requests.get(href)
    
    if response.status_code == 200:
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')
        
        res = soup.find('h3', {'id' : 'view_content'})
        content = res.text
        images = res.find_all('img', {'class' : 'lazy content-image'})

        if images != []:
            imageList = [url['data-original'] for url in res.find_all('img', {'class' : 'lazy content-image'})]
        else:
            imageList = []
        return content, imageList
    
    else:
        return np.nan, []    

In [ ]:
def parsing_save(re):
    re = BeautifulSoup(re, 'html.parser')
    category = re.find('a', {'class' : 'mw_basic_list_category'}).text
    date = re.find('div', {'class' : 'mw_basic_list_datetime media-no-text'}).text
    view = re.find('div', {'class' : 'mw_basic_list_hit media-no-text'}).text
    title = re.find('h3', {'class' : 'media-list-subject flex'}).text.replace('\n', '')
    rank = str(re.find('img')['src']).split('/')[-1].split('.')[0]
    href = re.find_all('a')[1]['href']
    try:
        writer = re.find('a', {'class' : 'sv_member'}).text.strip()
    except:
        writer = re.find('div', {'mw_basic_list_name media-no-text'}).text
    content, imageList = clawringContentImage(href)

    with open(save_path, 'a', newline = '') as csvfile:
        csvWriter = csv.DictWriter(csvfile, fieldnames = fieldNames)
        
        try:
            csvWriter.writerow({'title' : title,
                                'category' : category,
                                'date' : date,
                                'writer' : writer,
                                'rank' : rank,
                                'view' : view,
                                'href' : href,
                                'content' : content,
                                'imageList' : imageList})      
        except:
            print(href, ': Error')

In [ ]:
def clawringHref01(url, pageNum, startPage = 1):
    errorNum = 0

    if not os.path.isfile(save_path):
        tmp = pd.DataFrame(columns = fieldNames)
        tmp.to_csv(save_path, index = False)
        
        with open(save_path, 'w', newline = '') as csvfile:
            csvWriter = csv.DictWriter(csvfile, fieldnames = fieldNames)
            csvWriter.writeheader()

    for page in tqdm(range(startPage, pageNum + 1)):
        page = str(page)
        response = requests.get(url + page)
        if response.status_code == 200:
            html = response.text
            soup = BeautifulSoup(html, 'html.parser')
            re_list = soup.find_all('div', {"class":"list_wrap flex_spbtw"})            
            re_list = [str(re) for re in re_list]

            for re in re_list:
                parsing_save(re)

### 대다모 크롤링 대상
* 샴푸&두피케어
    * 샴푸&두피제품 찾기
    * 체험평가단
* 탈모톡톡
    * 탈모수다 : (완)
    * 샴푸&영양제 : (완)
* 탈모후기
    * 탈모치료성공스토리

1. 머리말 : category
2. 제목 : title
3. 작성자 : writer
4. 작성일 : date
5. 조회수 : view
6. 링크 : href
7. 계급 : rank

In [ ]:
path_save = 'href_탈모톡톡_탈모수다.csv'
path_save_base = '../Data'
save_path = os.path.join(path_save_base, path_save)
fieldNames = ['title', 'category', 'date', 'writer', 'rank', 'view', 'href', 'content', 'imageList']

#### 탈모톡톡_탈모수다

In [ ]:
url = 'https://daedamo.com/story?overlaps=1&page='
path_save = 'href_탈모톡톡_탈모수다.csv'
fieldNames = ['title', 'category', 'date', 'writer', 'rank', 'view', 'href', 'content', 'imageList']
pageNum = 2
startPage = 1
clawringHref01(url, pageNum, startPage)

#### 탈모톡톡_샴푸&영양제

In [ ]:
url = 'https://daedamo.com/balmo?overlaps=1&page='
path_save = 'href_탈모톡톡_샴푸&영양제.csv'
fieldNames = ['title', 'category', 'date', 'writer', 'rank', 'view', 'href', 'content', 'imageList']
pageNum = 120

clawringHref01(url, pageNum, path_save, fieldNames, startPage)

# 샴푸&두피케어-샴푸&두피케어
이거 같은 경우에는, 로그인을 해야하며, 

In [2]:
home_url = 'https://daedamo.com/new/'
login_url = 'https://daedamo.com/new/bbs/login.php'

login_info = {
    'mb_id' : 'dataflow28',
    'mb_password' : 'flowdata2833!!'
}

shamFind_url = 'https://daedamo.com/ingre?sca=%ED%83%88%EB%AA%A8%EA%B4%80%EB%A0%A8%EC%83%81%ED%92%88&bso=A&overlaps=5&page='
startPage = 1
endPage = 3

In [6]:
with requests.Session() as s:
    response = s.get(home_url)
    login_req = s.post(login_url, 
                       data = login_info)

    for page in range(startPage, endPage + 1):
        url = shamFind_url + str(page)
        response = s.get(url)
        
        if response.status_code == 200:
            html = response.text
            soup = BeautifulSoup(html, 'html.parser')
            re_list = soup.find_all('div', {'class' : 'bbs_list_box flex_spbtw'})
            
            for re in re_list:
                href = re.find('h3', {'class' : 'cont_subject'}).find('a')['href']
                view = re.find('div', {'class' : 'cont_info_hit'}).text

                response = s.get(href)
                
                if response.status_code == 200:
                    html = response.text
                    soup = BeautifulSoup(html, 'html.parser')
                    content = soup.find_all('p', {'class' : 'cont'})
                    howToUse = content[0].text
                    ingredients = content[1].text
                    imgage = soup.find('img', {'class' : 'lazy'})['src']
                    tag = soup.find('div', {'class' : 'tag'}).text
                    title = soup.find('div', {'class' : 'subject'}).text
                    brand = soup.find('div', {'class' : 'brand'}).text
                    company = soup.find('div', {'class' : 'company'}).text
                    
                    volumeAndPrice = soup.find('div', {'class' : 'volume_price'}).text.strip().split('\n')
                    volume = volumeAndPrice[0]
                    price = volumeAndPrice[1]
                    reviewNum = soup.find('div', {'class' : 'score_title'}).text
                    
                    
                    # best review
                    # 리뷰가 없을 수 있으니 해당 내용 예외 처리 필요
                    bestReview = soup.find('div', {'class' : 'comment-item comment-best'})
                    if pd.isnull(bestReview):
                        bestReview = np.nan
                        bestReviewDate = np.nan
                        bestReviewProduct = np.nan
                        bestReviewPrice = np.nan
                        bestReviewRebuy = np.nan
                    else:
                        bestReviewer = bestReview.find('span', {'class' : 'sv_wrap'}).text
                        bestReviewDate = bestReview.find('span', {'class' : 'datetime'}).text
    #                     bestReviewProduct = bestReview.find_all('div', {'class' : 'score'})
                        print(bestReview.find_all('div', {'class' : 'score'}))
                    
                    
                    # comment.ingre.ajax.php
#                     pg_current = soup.find('strong', {'class' : 'pg_current'}).text
                    pg = soup.find('div', {'class' : 'mw_basic_comment_page'})
                    pg_current = pg.find('strong', {'class' : 'pg_current'})
                    print(pg)
                    print(pg_current)
                    # pg_page pg_next
                    # 여기에 조건을 넣어야 한다.
                    pg_next = soup.find_all('a', {'class' : 'pg_page pg_next'})
                    print(pg_next)
#                     if (len(pg_next) != 0) & 
                    
        
        else:
            print('error')
            continue

<div class="mw_basic_comment_page"></div>
None
[]
<div class="mw_basic_comment_page"></div>
None
[]
<div class="mw_basic_comment_page"></div>
None
[]
<div class="mw_basic_comment_page"></div>
None
[]
<div class="mw_basic_comment_page"></div>
None
[]
<div class="mw_basic_comment_page"></div>
None
[]
<div class="mw_basic_comment_page"></div>
None
[]
<div class="mw_basic_comment_page"></div>
None
[]
<div class="mw_basic_comment_page"></div>
None
[]
<div class="mw_basic_comment_page"></div>
None
[]
<div class="mw_basic_comment_page"></div>
None
[]


KeyboardInterrupt: 